In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import requests
import json
from pprint import pprint

In [2]:
plotly.tools.set_credentials_file(username='wangdian', api_key='DKvDkKjnNZWzlT6zhaCq')
mapbox_access_token = 'pk.eyJ1Ijoid2FuZ2RpYW5kaWFuIiwiYSI6ImNqb3JyeWV4czAyeWwzcW85enRyZmp5dm8ifQ.UIVacLewjXi3XRL2XvlsgA'

In [3]:
# map plot location
path = 'data/MCI_2014_to_2017.csv'

df = pd.read_csv(path)
df.head()

,X,Y,Index_,event_unique_id,occurrencedate,reporteddate,premisetype,ucr_code,ucr_ext,offence,...,occurrencedayofyear,occurrencedayofweek,occurrencehour,MCI,Division,Hood_ID,Neighbourhood,Lat,Long,FID
0,-79.520401,43.768829,14601,GO-20142775022,2014-08-25T04:00:00.000Z,2014-08-25T04:00:00.000Z,Outside,1430,100,Assault,...,237.0,Monday,18,Assault,D31,24,Black Creek (24),43.768829,-79.520401,14001
1,-79.580856,43.642574,14602,GO-20142870874,2014-08-25T04:00:00.000Z,2014-09-08T04:00:00.000Z,House,2120,220,B&E W'Intent,...,237.0,Monday,9,Break and Enter,D22,11,Eringate-Centennial-West Deane (11),43.642574,-79.580856,14002
2,-79.260445,43.762909,14603,GO-20142802386,2014-08-25T04:00:00.000Z,2014-08-29T04:00:00.000Z,House,1430,100,Assault,...,237.0,Monday,11,Assault,D41,127,Bendale (127),43.762909,-79.260445,14003
3,-79.367546,43.663208,14604,GO-20142777955,2014-08-25T04:00:00.000Z,2014-08-26T04:00:00.000Z,Commercial,2120,200,B&E,...,237.0,Monday,17,Break and Enter,D51,71,Cabbagetown-South St.James Town (71),43.663208,-79.367546,14004
4,-79.231758,43.776440,14605,GO-20142778699,2014-08-25T04:00:00.000Z,2014-08-26T04:00:00.000Z,Other,2120,200,B&E,...,237.0,Monday,16,Break and Enter,D43,137,Woburn (137),43.776440,-79.231758,14005


In [4]:
df = df.sort_values(by=['Hood_ID'])
x = df.loc[df['Hood_ID'] == 1]['X'].mean()

In [5]:
map_id_dict = {
    'hood_id': df['Hood_ID'].unique(),
    'lon': df.groupby('Hood_ID')['X'].mean(),
    'lan': df.groupby('Hood_ID')['Y'].mean(),
    'neighbourhood': df.groupby('Hood_ID')['Neighbourhood'].unique()
}

map_id_df = pd.DataFrame(map_id_dict, index=map_id_dict['hood_id'])
map_id_df.head()

,hood_id,lan,lon,neighbourhood
1,1,43.719981,-79.598015,[West Humber-Clairville (1)]
2,2,43.745059,-79.588351,[Mount Olive-Silverstone-Jamestown (2)]
3,3,43.738668,-79.568820,[Thistletown-Beaumond Heights (3)]
4,4,43.720970,-79.566224,[Rexdale-Kipling (4)]
5,5,43.720902,-79.551941,[Elms-Old Rexdale (5)]


In [6]:
# total crime
path = 'data/crime_totals_per_hood.csv'

crime_total_df = pd.read_csv(path, header=None)
crime_total_df.columns = ['hood_id', 'total_crime']
crime_total_df.head()

,hood_id,total_crime
0,1,3264
1,2,1806
2,3,475
3,4,509
4,5,444


In [7]:
crime_total_df = crime_total_df.merge(map_id_df, on='hood_id', how='outer' )
crime_total_df.head()

,hood_id,total_crime,lan,lon,neighbourhood
0,1,3264,43.719981,-79.598015,[West Humber-Clairville (1)]
1,2,1806,43.745059,-79.588351,[Mount Olive-Silverstone-Jamestown (2)]
2,3,475,43.738668,-79.568820,[Thistletown-Beaumond Heights (3)]
3,4,509,43.720970,-79.566224,[Rexdale-Kipling (4)]
4,5,444,43.720902,-79.551941,[Elms-Old Rexdale (5)]


In [8]:
crime_total_map_plot = []
for index, row in crime_total_df.iterrows():
    
    data = go.Scattermapbox(
            lat=[str(row['lan'])],
            lon=[str(row['lon'])],
            mode='markers',
            marker=dict(
                size=0.02 * row['total_crime'],
                opacity= 0.7,
                color = 'rgb(17, 157, 255)'
            ),
            
            name=row['neighbourhood'][0],
            text=row['neighbourhood'][0] # pending disctrict name
        )
    
    crime_total_map_plot.append(data)
    
layout = go.Layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=43.7,
            lon=-79.5
        ),
        pitch=0,
        zoom=10
    ),
)

fig = dict(data=crime_total_map_plot, layout=layout)

py.iplot(fig, filename='Toronto Crime')

In [9]:
crime_total_df['total_crime'].max()

4685

In [10]:
crime_total_df['total_crime'].min()

146

In [60]:
bins = [0, 1000, 2000, 3000, 4000, 5000]
bin_label = ['safe <1000', 'low 1000-2000', 'average 2000-3000', 'high 3000-4000', 'danger >4000' ]
bin_color = ['green', 'blue', 'gold', 'orange', 'pink']
#add label to crime_total_df
crime_total_df['label'] = pd.cut(crime_total_df['total_crime'], bins, labels=bin_label)
crime_total_df['color'] = pd.cut(crime_total_df['total_crime'], bins, labels=bin_color)

In [61]:
crime_total_df.head()

,hood_id,total_crime,lan,lon,neighbourhood,label,color
0,1,3264,43.719981,-79.598015,[West Humber-Clairville (1)],high 3000-4000,orange
1,2,1806,43.745059,-79.588351,[Mount Olive-Silverstone-Jamestown (2)],low 1000-2000,blue
2,3,475,43.738668,-79.568820,[Thistletown-Beaumond Heights (3)],safe <1000,green
3,4,509,43.720970,-79.566224,[Rexdale-Kipling (4)],safe <1000,green
4,5,444,43.720902,-79.551941,[Elms-Old Rexdale (5)],safe <1000,green


In [72]:
df = crime_total_df.loc[crime_total_df['color'] == 'pink']
df

,hood_id,total_crime,lan,lon,neighbourhood,label,color
74,75,4685,43.658719,-79.380407,[Church-Yonge Corridor (75)],danger >4000,pink
76,77,4328,43.644643,-79.388908,[Waterfront Communities-The Island (77)],danger >4000,pink


In [78]:
df['label'].iloc[0]

'danger >4000'

In [79]:
#creat data list
crime_data_list = []

for color in bin_color:
    df = crime_total_df.loc[crime_total_df['color'] == color]
    
    lon_list = []
    lat_list = []
    crime_list = []
    neighbour_list = []
    for index, row in df.iterrows():
        lon_list.append(row['lon'])
        lat_list.append(row['lan'])
        crime_list.append(row['total_crime'] / 77)
        neighbour_list.append(row['neighbourhood'][0])
        
        
    data = go.Scattermapbox(
        lon = lon_list,
        lat = lat_list,
        #text = df_sub['neighbourhood'][0],
        marker = dict(
            size = crime_list,
            # sizeref = 2. * max(df_sub['pop']/scale) / (25 ** 2),
            color = color,
            opacity = 0.7
        ),
        name=df['label'].iloc[0],
        text=neighbour_list
    )
    crime_data_list.append(data)

crime_data_list[0]

Scattermapbox({
    'lat': [43.73866841066956, 43.72097002490198, 43.72090165470848,
            43.696529879487294, 43.68181599471874, 43.690981623333414,
            43.66927492774869, 43.66744382483218, 43.65610530192642,
            43.63334217446808, 43.6441560550135, 43.65129019620589,
            43.632507549801325, 43.6003281441998, 43.592971617045485,
            43.60531714712328, 43.74421875744953, 43.71704761005813,
            43.70966525664337, 43.71279853462597, 43.69941508320616,
            43.72143037450143, 43.73939031506846, 43.76145981952299,
            43.781682932729936, 43.785116903836624, 43.77119463875603,
            43.75078041639997, 43.72712344086863, 43.75428904944906,
            43.73052051176475, 43.7341916360809, 43.724862848319326,
            43.71527563867778, 43.78613933927491, 43.79899161854686,
            43.79621059476753, 43.78792814298524, 43.7724466412577,
            43.77207402473407, 43.70571030826022, 43.7061711682566,
            43.7

In [83]:
acc_layout = go.Layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=43.7,
            lon=-79.5
        ),
        pitch=0,
        zoom=10,
        style='light'
    ),
)
    
fig = dict(data=crime_data_list, layout=acc_layout)

py.iplot(fig, filename='Toronto Accnum')